In [448]:
#packages needed to do the project. 
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import requests
import os
import io

# Gather: 

### Data Sources: 
For this project, we will use three different data sources. 
    1. The WeRateDogs Twitter archive, which is provided by WeRateDog to Udacity via a csv file. 
    2. The Tweet Image prediction, which we will have to download programmatically using Python's requests library. 
    3. Each Tweet's retweet and favorite ('like') count, which we will download from Twitter using Tweepy a twitter API. 
    

##### Enhanced Twitter Archive:
- The WeRateDogs Twitter archive contains basic tweet data for all 5000+ of their tweets, but not everything. One column the archive does contain though: each tweet's text, which I used to extract rating, dog name, and dog "stage" (i.e. doggo, floofer, pupper, and puppo) to make this Twitter archive "enhanced." Of the 5000+ tweets, I have filtered for tweets with ratings only (there are 2356).

In [552]:

twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [553]:
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [554]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

##### Tweet Image prediction: 

We will use Python's request and io libraries to download this dataset from a Udacity hosted server. 

In [5]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url).content
image_pred = pd.read_csv(io.StringIO(response.decode('utf-8')), sep='\t')

In [42]:
#Save the file in a csv format. 
#image_pred.to_csv('image-predictions.csv', index=False)


In [6]:
image_pred.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
445,674646392044941312,https://pbs.twimg.com/media/CVzTUGrW4AAirJH.jpg,1,flat-coated_retriever,0.837448,True,groenendael,0.086166,True,Labrador_retriever,0.016052,True
190,669393256313184256,https://pbs.twimg.com/media/CUopnHPVEAAcL2o.jpg,1,cocker_spaniel,0.359843,True,Blenheim_spaniel,0.139519,True,toy_poodle,0.132746,True
756,688519176466644993,https://pbs.twimg.com/media/CY4ciRFUMAAovos.jpg,1,Pembroke,0.696372,True,Cardigan,0.121052,True,Shetland_sheepdog,0.050592,True
491,675534494439489536,https://pbs.twimg.com/media/CV_7CV6XIAEV05u.jpg,1,chow,0.749368,True,schipperke,0.133738,True,Newfoundland,0.049914,True
564,678255464182861824,https://pbs.twimg.com/media/CWmlvxJU4AEAqaN.jpg,1,Chihuahua,0.613819,True,Yorkshire_terrier,0.127931,True,Pomeranian,0.062124,True


In [22]:
image_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 119.6+ KB


Additional Data via the Twitter API

- Back to the basic-ness of Twitter archives: retweet count and favorite count are two of the notable column omissions. Fortunately, this additional data can be gathered by anyone from Twitter's API. Well, "anyone" who has access to data for the 3000 most recent tweets, at least. But you, because you have the WeRateDogs Twitter archive and specifically the tweet IDs within it, can gather this data for all 5000+. And guess what? You're going to query Twitter's API to gather this valuable data.

In [10]:
#Consumer Key and Scecret Codes: 
consumer_key = 'sIxqqO3GYMtb19GPfnYT9m1jJ'
consumer_secret = 'F6tUkGIhSXVyLtjyi9zdtbTQ8IbEbaFAHUb4Mvg68leQYKALRO'
access_token= '1044734987319205888-wBxtFZXiXTcyDx2WkON4rB5TapfdoY'
access_token_secret = 'tNZPiIsHXoe9LVwdOg4u8SGY61WoqkPg1VOZcjWYvnQLx'

In [11]:
#Authenticating my twitter account through tweepy. 
import tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [12]:
#to check if our API is working: 
user = api.me()
print (user.name)

nfaheem


In [38]:
import time
start_time = time.time()

tweet_id_list = (twitter_archive['tweet_id'])
twitter_dict = [] 
deleted_tweets =[]
for tweet_id in tweet_id_list:
    try:
        tweet = api.get_status(tweet_id, tweet_mode='extended',
                               wait_on_rate_limit=True,
                               wait_on_rate_limit_notify=True)
        
        #getting the full text. 
        full_text = tweet._json['full_text']
        #getting the retweet_count for the user id: 
        retweet_count = tweet._json['retweet_count']
        #getting the favorite ('like') count for each tweet. 
        favorite_count = tweet._json['favorite_count']
        
        twitter_dict.append({'id': tweet_id, 
                            'full_text' : full_text, 
                            'retweet_count': retweet_count, 
                            'favorite_count' : favorite_count})
    except: 
        deleted_tweets.append(tweet_id)
    

print("\nThis took %s seconds." % (time.time() - start_time))

Rate limit reached. Sleeping for: 284
Rate limit reached. Sleeping for: 702



This took 1525.8535025119781 seconds.


In [555]:
#Converting the dictionary into a Pandas Dataframe. 
twitter_json_raw = pd.DataFrame(twitter_dict, columns = ['id', 'full_text', 'retweet_count', 'favorite_count'])            


In [556]:
twitter_json_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341 entries, 0 to 2340
Data columns (total 4 columns):
id                2341 non-null int64
full_text         2341 non-null object
retweet_count     2341 non-null int64
favorite_count    2341 non-null int64
dtypes: int64(3), object(1)
memory usage: 64.1+ KB


In [557]:
#Save the file in a csv format. 
twitter_json_raw.to_csv('twitter_json_raw.csv', index=False)

In [558]:
#opening the saved Json file in csv: 
twitter_json = pd.read_csv('twitter_json_raw.csv')

In [560]:
twitter_json.head()

,id,full_text,retweet_count,favorite_count
0,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,8431,38383
1,892177421306343426,This is Tilly. She's just checking pup on you....,6211,32890
2,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,4111,24785
3,891689557279858688,This is Darla. She commenced a snooze mid meal...,8549,41722
4,891327558926688256,This is Franklin. He would like you to stop ca...,9284,39898


# Assess: 

In [561]:
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [54]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [109]:
twitter_archive.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [117]:
#Checking some of the higher numerators to get their index: 
print(twitter_archive.loc[twitter_archive.rating_numerator == 1776, 'text'])
print(twitter_archive.loc[twitter_archive.rating_numerator == 960, 'text']) 
print(twitter_archive.loc[twitter_archive.rating_numerator == 666, 'text']) 
print(twitter_archive.loc[twitter_archive.rating_numerator == 204, 'text'])
print(twitter_archive.loc[twitter_archive.rating_numerator == 182, 'text']) 
print(twitter_archive.loc[twitter_archive.rating_numerator == 204, 'text']) 
print(twitter_archive.loc[twitter_archive.rating_numerator == 165, 'text']) 
print(twitter_archive.loc[twitter_archive.rating_numerator == 144, 'text']) 
print(twitter_archive.loc[twitter_archive.rating_numerator == 143, 'text']) 
print(twitter_archive.loc[twitter_archive.rating_numerator == 121, 'text']) 

979    This is Atticus. He's quite simply America af....
Name: text, dtype: object
313    @jonnysun @Lin_Manuel ok jomny I know you're e...
Name: text, dtype: object
189    @s8n You tried very hard to portray this good ...
Name: text, dtype: object
1120    Say hello to this unbelievably well behaved sq...
Name: text, dtype: object
290    @markhoppus 182/10
Name: text, dtype: object
1120    Say hello to this unbelievably well behaved sq...
Name: text, dtype: object
902    Why does this never happen at my front door......
Name: text, dtype: object
1779    IT'S PUPPERGEDDON. Total of 144/120 ...I think...
Name: text, dtype: object
1634    Two sneaky puppers were not initially seen, mo...
Name: text, dtype: object
1635    Someone help the girl is being mugged. Several...
Name: text, dtype: object


In [118]:
#Printing all the higher numerators' text to make sure they are accurate
print(twitter_archive['text'][979]) #accurate 
print(twitter_archive['text'][313]) #This is going to be fixed. 
print(twitter_archive['text'][189]) #accurate
print(twitter_archive['text'][1120]) #accurate
print(twitter_archive['text'][290]) #accurate
print(twitter_archive['text'][902]) #accurate
print(twitter_archive['text'][1779]) #accurate
print(twitter_archive['text'][1634]) #accurate
print(twitter_archive['text'][1635]) #accurate

This is Atticus. He's quite simply America af. 1776/10 https://t.co/GRXwMxLBkh
@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho
@s8n You tried very hard to portray this good boy as not so good, but you have ultimately failed. His goodness shines through. 666/10
Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv
@markhoppus 182/10
Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE
IT'S PUPPERGEDDON. Total of 144/120 ...I think https://t.co/ZanVtAtvIq
Two sneaky puppers were not initially seen, moving the rating to 143/130. Please forgive us. Thank you https://t.co/kRK51Y5ac3
Someone help the girl is being mugged. Several are distracting her while two steal her shoes. Clever puppers 121/110 https://t.co/1zfnTJLt55


In [119]:
#checking to see see if the the floating numerators are parsed well. 
with pd.option_context('max_colwidth', 200):
    display(twitter_archive[twitter_archive['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

c:\users\nasee\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


,tweet_id,text,rating_numerator,rating_denominator
45,883482846933004288,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",5,10
340,832215909146226688,"RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…",75,10
695,786709082849828864,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",75,10
763,778027034220126208,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,27,10
1689,681340665377193984,I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,5,10
1712,680494726643068929,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,26,10


In [67]:
twitter_archive.name.value_counts()

None         745
a             55
Charlie       12
Oliver        11
Lucy          11
Cooper        11
Tucker        10
Lola          10
Penny         10
Bo             9
Winston        9
the            8
Sadie          8
an             7
Toby           7
Daisy          7
Buddy          7
Bailey         7
Jack           6
Dave           6
Oscar          6
Milo           6
Leo            6
Rusty          6
Bella          6
Stanley        6
Scout          6
Koda           6
Jax            6
Chester        5
            ... 
Ebby           1
Jeremy         1
Yukon          1
Dug            1
Skittle        1
Sparky         1
Kathmandu      1
Gòrdón         1
Colin          1
Jo             1
Crouton        1
Bluebert       1
Mosby          1
Izzy           1
Mojo           1
Goliath        1
Tanner         1
Naphaniel      1
Aja            1
Kona           1
Bayley         1
Benny          1
Jaspers        1
Brownie        1
Shadoe         1
Sundance       1
Mauve          1
Ralphy        

In [186]:
#unusual names
twitter_archive[twitter_archive['name'].apply(len) < 3]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
56,881536004380872706,NaN,NaN,2017-07-02 15:32:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a pupper approaching maximum borkdrive...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881536004...,14,10,a,None,None,pupper,None
393,825876512159186944,NaN,NaN,2017-01-30 01:21:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Mo. No one will push him around in the...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/825876512...,11,10,Mo,None,None,None,None
446,819015337530290176,NaN,NaN,2017-01-11 02:57:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Bo. He was a very good ...,8.190048e+17,4.196984e+09,2017-01-11 02:15:36 +0000,https://twitter.com/dog_rates/status/819004803...,14,10,Bo,doggo,None,None,None
449,819004803107983360,NaN,NaN,2017-01-11 02:15:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bo. He was a very good First Doggo. 14...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/819004803...,14,10,Bo,doggo,None,None,None
553,804026241225523202,NaN,NaN,2016-11-30 18:16:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bo. He's going to make me cry. 13/10 p...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/804026241...,13,10,Bo,None,None,None,None
583,800188575492947969,NaN,NaN,2016-11-20 04:06:37 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Bo. He's a Benedoop Cum...,6.816941e+17,4.196984e+09,2015-12-29 04:31:49 +0000,https://twitter.com/dog_rates/status/681694085...,11,10,Bo,None,None,pupper,None
649,792913359805018113,NaN,NaN,2016-10-31 02:17:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a perfect example of someone who has t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/792913359...,13,10,a,None,None,None,None
679,789137962068021249,NaN,NaN,2016-10-20 16:15:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bo. He's a West Congolese Bugaboop Snu...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/789137962...,12,10,Bo,None,None,None,None
686,788070120937619456,NaN,NaN,2016-10-17 17:32:13 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Bo and Ty. Bo eats pape...,7.610045e+17,4.196984e+09,2016-08-04 01:03:17 +0000,https://twitter.com/dog_rates/status/761004547...,11,10,Bo,None,None,None,None
759,778396591732486144,NaN,NaN,2016-09-21 00:53:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is an East African Chalupa...,7.030419e+17,4.196984e+09,2016-02-26 02:20:37 +0000,https://twitter.com/dog_rates/status/703041949...,10,10,an,None,None,None,None


In [108]:
twitter_archive[twitter_archive.name == 'a']

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
56,881536004380872706,NaN,NaN,2017-07-02 15:32:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a pupper approaching maximum borkdrive...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881536004...,14,10,a,None,None,pupper,None
649,792913359805018113,NaN,NaN,2016-10-31 02:17:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a perfect example of someone who has t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/792913359...,13,10,a,None,None,None,None
801,772581559778025472,NaN,NaN,2016-09-04 23:46:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Guys this is getting so out of hand. We only r...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/772581559...,10,10,a,None,None,None,None
1002,747885874273214464,NaN,NaN,2016-06-28 20:14:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a mighty rare blue-tailed hammer sherk...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/747885874...,8,10,a,None,None,None,None
1004,747816857231626240,NaN,NaN,2016-06-28 15:40:07 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Viewer discretion is advised. This is a terrib...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/747816857...,4,10,a,None,None,None,None
1017,746872823977771008,NaN,NaN,2016-06-26 01:08:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a carrot. We only rate dogs. Please on...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/746872823...,11,10,a,None,None,None,None
1049,743222593470234624,NaN,NaN,2016-06-15 23:24:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a very rare Great Alaskan Bush Pupper....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/743222593...,12,10,a,None,None,pupper,None
1193,717537687239008257,NaN,NaN,2016-04-06 02:21:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",People please. This is a Deadly Mediterranean ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/717537687...,11,10,a,None,None,None,None
1207,715733265223708672,NaN,NaN,2016-04-01 02:51:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a taco. We only rate dogs. Please only...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/715733265...,10,10,a,None,None,None,None
1340,704859558691414016,NaN,NaN,2016-03-02 02:43:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a heartbreaking scene of an incredible...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/704859558...,10,10,a,None,None,pupper,None


In [106]:
twitter_archive[twitter_archive['retweeted_status_id'].notnull()== True]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Canela. She attempted s...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,8.860537e+17,1.960740e+07,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Lilly. She just paralle...,8.305833e+17,4.196984e+09,2017-02-12 01:04:29 +0000,https://twitter.com/dog_rates/status/830583320...,13,10,Lilly,None,None,None,None
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Emmy. She was adopted t...,8.780576e+17,4.196984e+09,2017-06-23 01:10:23 +0000,https://twitter.com/dog_rates/status/878057613...,14,10,Emmy,None,None,None,None
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Shadow. In an attempt to r...,8.782815e+17,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitt...",13,10,Shadow,None,None,None,None
74,878316110768087041,NaN,NaN,2017-06-23 18:17:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Terrance. He's being yelle...,6.690004e+17,4.196984e+09,2015-11-24 03:51:38 +0000,https://twitter.com/dog_rates/status/669000397...,11,10,Terrance,None,None,None,None
78,877611172832227328,NaN,NaN,2017-06-21 19:36:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @rachel2195: @dog_rates the boyfriend and h...,8.768508e+17,5.128045e+08,2017-06-19 17:14:49 +0000,https://twitter.com/rachel2195/status/87685077...,14,10,None,None,None,pupper,None
91,874434818259525634,NaN,NaN,2017-06-13 01:14:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Coco. At first I though...,8.663350e+17,4.196984e+09,2017-05-21 16:48:45 +0000,https://twitter.com/dog_rates/status/866334964...,12,10,Coco,None,None,None,None
95,873697596434513921,NaN,NaN,2017-06-11 00:25:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Walter. He won't start ...,8.688804e+17,4.196984e+09,2017-05-28 17:23:24 +0000,https://twitter.com/dog_rates/status/868880397...,14,10,Walter,None,None,None,None
97,873337748698140672,NaN,NaN,2017-06-10 00:35:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Sierra. She's one preci...,8.732138e+17,4.196984e+09,2017-06-09 16:22:42 +0000,https://www.gofundme.com/help-my-baby-sierra-g...,12,10,Sierra,None,None,pupper,None


In [125]:
#Although the jpb_url is duplicated, but the rest of the columns are unique, including tweet_id which is our main identifier of a tweet
image_pred[image_pred.jpg_url.duplicated()]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1297,752309394570878976,https://pbs.twimg.com/ext_tw_video_thumb/67535...,1,upright,0.303415,False,golden_retriever,0.181351,True,Brittany_spaniel,0.162084,True
1315,754874841593970688,https://pbs.twimg.com/media/CWza7kpWcAAdYLc.jpg,1,pug,0.272205,True,bull_mastiff,0.251530,True,bath_towel,0.116806,False
1333,757729163776290825,https://pbs.twimg.com/media/CWyD2HGUYAQ1Xa7.jpg,2,cash_machine,0.802333,False,schipperke,0.045519,True,German_shepherd,0.023353,True
1345,759159934323924993,https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg,1,Irish_terrier,0.254856,True,briard,0.227716,True,soft-coated_wheaten_terrier,0.223263,True
1349,759566828574212096,https://pbs.twimg.com/media/CkNjahBXAAQ2kWo.jpg,1,Labrador_retriever,0.967397,True,golden_retriever,0.016641,True,ice_bear,0.014858,False
1364,761371037149827077,https://pbs.twimg.com/tweet_video_thumb/CeBym7...,1,brown_bear,0.713293,False,Indian_elephant,0.172844,False,water_buffalo,0.038902,False
1368,761750502866649088,https://pbs.twimg.com/media/CYLDikFWEAAIy1y.jpg,1,golden_retriever,0.586937,True,Labrador_retriever,0.398260,True,kuvasz,0.005410,True
1387,766078092750233600,https://pbs.twimg.com/media/ChK1tdBWwAQ1flD.jpg,1,toy_poodle,0.420463,True,miniature_poodle,0.132640,True,Chesapeake_Bay_retriever,0.121523,True
1407,770093767776997377,https://pbs.twimg.com/media/CkjMx99UoAM2B1a.jpg,1,golden_retriever,0.843799,True,Labrador_retriever,0.052956,True,kelpie,0.035711,True
1417,771171053431250945,https://pbs.twimg.com/media/CVgdFjNWEAAxmbq.jpg,3,Samoyed,0.978833,True,Pomeranian,0.012763,True,Eskimo_dog,0.001853,True


In [71]:
image_pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [75]:
image_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 119.6+ KB


In [80]:
image_pred.p1_dog.value_counts()

True     1532
False     543
Name: p1_dog, dtype: int64

In [83]:
twitter_json.head()

,id,full_text,retweet_count,favorite_count
0,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,8431,38383
1,892177421306343426,This is Tilly. She's just checking pup on you....,6211,32890
2,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,4111,24785
3,891689557279858688,This is Darla. She commenced a snooze mid meal...,8549,41722
4,891327558926688256,This is Franklin. He would like you to stop ca...,9284,39898


In [84]:
twitter_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341 entries, 0 to 2340
Data columns (total 4 columns):
id                2341 non-null int64
full_text         2341 non-null object
retweet_count     2341 non-null int64
favorite_count    2341 non-null int64
dtypes: int64(3), object(1)
memory usage: 64.1+ KB


In [85]:
twitter_json.retweet_count.describe()

count     2341.000000
mean      2967.859889
std       4973.295285
min          0.000000
25%        597.000000
50%       1386.000000
75%       3466.000000
max      84504.000000
Name: retweet_count, dtype: float64

In [88]:
twitter_json.favorite_count.describe()

count      2341.000000
mean       8022.877830
std       12351.271051
min           0.000000
25%        1390.000000
50%        3504.000000
75%        9855.000000
max      164927.000000
Name: favorite_count, dtype: float64

#### Quality:

##### twiter_archive table: 
- name has wrong and faulty namessuch as a, an, the, light, life, by, actually, old, all, getting, just, this, mad, not, very, one, my, quite and such in `twitter_archive_clean` table
- timestamp is a string object not datetime object
- organize the date format to mm dd yyyy format and time format to HH MM format. 
- 181 tweets are in the retweet_status id and need to be dropped.
- 78 tweets are replys that need to be dropped. 
- change the datatype of the stage column to categorical. 
- numerator has the following wrong becuase it did not parse the float numerators well. 
    - 786709082849828864 : 75 instaed of 9.75
    - 680494726643068929 : 26 inplace of 11.26
    - 778027034220126208 : 27 inplace of 11.27
    - 883482846933004288 : 5  instead of 13.5
    - 681340665377193984 : 5 instead 9.5
    - 680494726643068929 : 26 instead of 11.26
    - 835246439529840640 : 960 intead of 13
    - 716439118184652801 : 50 instaed of  11
    - 666287406224695296 : 1 instaed of 9
    
- denominator of following is wrong: 
    - 835246439529840640 : 0 intead of 10
    - 716439118184652801 : 50 instaed of  10
    - 666287406224695296 : 2 instead of 10
 - some tweets have denominators other than 10. 
    
##### image_pred table: 

- replace "-" to " " in p1, p2 and p3
- Capitalize the name the names in p1, p2, and p3. 
- drop rows that does not have a dog breed.

##### twitter_json table: 
- rename the id column to tweet_id to match the other tables.
- tweet_id (838085839343206401) is not about a dog and has 0 retweets. 



#### Tidiness: 
- split the timestamp column in date and time colums in `twitter_archive` table
- doggo, floofer, pupper and puppo should be one column 'stage' in `twitter_archive` table
- excessive columns: in_reply_to_status_id, in_reply_to_user_id, source, retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp, timestamp in `twitter_archive` table. 
- full_text column also exits in twitter_archive, so it needs to be dropped from `twitter_json` table.
- merge the the dog breed (p1-p3) columns. Have only one prediction and its confidence interval in `image_pred` table 
- remove columns (img_num-p3_dog) as they will not be needed for the analysis in `image_pred` table 
- twiter_archive and twitter_json should be merged with image_pred table. 

# Clean

In [562]:
twitter_archive_clean = twitter_archive.copy()
image_pred_clean = image_pred.copy()
twitter_json_clean = twitter_json.copy()

## Quality:
###### name has wrong and faulty namessuch as a, an, the, light, life, by, actually, old, all, getting, just, this, mad, not, very, one, my, quite and such in `twitter_archive_clean` table

### Define
replace the above words to none because those words are not dog names. We could do this by creating a for loop to save time and adding all the names that we want to replace in a list. 



### Code

In [563]:
names_to_be_replace = ['a','an','the','light','life','by','actually','old','all','getting',
                      'just','this','mad','not','very','one','my','quite','such']
for name in names_to_be_replace:
    twitter_archive_clean['name'].replace(name, 'None', inplace=True)

### Test

In [564]:
twitter_archive_clean.name.value_counts()[:10]

None       848
Charlie     12
Oliver      11
Cooper      11
Lucy        11
Tucker      10
Lola        10
Penny       10
Bo           9
Winston      9
Name: name, dtype: int64

In [565]:
twitter_archive_clean.sample(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1726,680055455951884288,NaN,NaN,2015-12-24 16:00:30 +0000,"<a href=""https://about.twitter.com/products/tw...","Meet Sammy. At first I was like ""that's a snow...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/680055455...,10,10,Sammy,None,None,None,None
112,870804317367881728,NaN,NaN,2017-06-03 00:48:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Real funny guys. Sending in a pic without a do...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/870804317...,11,10,None,None,None,None,None
1366,702671118226825216,NaN,NaN,2016-02-25 01:47:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Rambo &amp; Kiwi. Rambo's the pup with th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/702671118...,10,10,Rambo,None,None,None,None


###### - timestamp is a string object not datetime object in the `twitter_archive_clean` table
###### - split the timestamp column in date and time colums. 
###### - organize the date formate to mm dd yyyy format



### Define
Convert the timestamp datatype to datetime using panda's, then split the column into two.

### Code

In [566]:
twitter_archive_clean.timestamp = pd.to_datetime(twitter_archive_clean.timestamp)

In [567]:
#Splitting the timestamp column into date and time. 
dates, times = zip(*[(d.date(), d.time()) for d in twitter_archive_clean['timestamp']])
twitter_archive_clean = twitter_archive_clean.assign(date=dates, time=times)

In [568]:
#changing the date and time format: 
twitter_archive_clean['date'] = twitter_archive_clean['timestamp'].dt.strftime('%m/%d/%Y')
twitter_archive_clean['time'] = twitter_archive_clean['timestamp'].dt.strftime('%H:%M')

In [314]:
#convert the date and time format again since the previous operation converted the format to string object. 
#twitter_archive_clean['date'] = pd.to_datetime(twitter_archive_clean['date'])
#twitter_archive_clean['time'] = pd.to_datetime(twitter_archive_clean['time'])

### Test

In [569]:
twitter_archive_clean.sample(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,date,time
1267,709566166965075968,NaN,NaN,2016-03-15 02:25:31,"<a href=""http://twitter.com/download/iphone"" r...",This is Olaf. He's gotta be rare. Seems sturdy...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/709566166...,12,10,Olaf,None,None,None,None,03/15/2016,02:25
1646,683834909291606017,NaN,NaN,2016-01-04 02:18:42,"<a href=""http://twitter.com/download/iphone"" r...",Here we see a faulty pupper. Might need to rep...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/683834909...,9,10,None,None,None,pupper,None,01/04/2016,02:18


###### - 181 tweets are in the retweet_status id and need to be dropped. 


### Define
- Exclude tweets which are in the retweeted status, as we are only concerned about the main tweets in this project. We could do so by using notnull() and then drop them. 
- Identify and exclude tweets tat are in reply status. 

### Code

In [570]:
#Identifying retweets: 
twitter_archive[twitter_archive['retweeted_status_id'].notnull()== True]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Canela. She attempted s...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,8.860537e+17,1.960740e+07,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Lilly. She just paralle...,8.305833e+17,4.196984e+09,2017-02-12 01:04:29 +0000,https://twitter.com/dog_rates/status/830583320...,13,10,Lilly,None,None,None,None
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Emmy. She was adopted t...,8.780576e+17,4.196984e+09,2017-06-23 01:10:23 +0000,https://twitter.com/dog_rates/status/878057613...,14,10,Emmy,None,None,None,None
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Shadow. In an attempt to r...,8.782815e+17,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitt...",13,10,Shadow,None,None,None,None
74,878316110768087041,NaN,NaN,2017-06-23 18:17:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Terrance. He's being yelle...,6.690004e+17,4.196984e+09,2015-11-24 03:51:38 +0000,https://twitter.com/dog_rates/status/669000397...,11,10,Terrance,None,None,None,None
78,877611172832227328,NaN,NaN,2017-06-21 19:36:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @rachel2195: @dog_rates the boyfriend and h...,8.768508e+17,5.128045e+08,2017-06-19 17:14:49 +0000,https://twitter.com/rachel2195/status/87685077...,14,10,None,None,None,pupper,None
91,874434818259525634,NaN,NaN,2017-06-13 01:14:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Coco. At first I though...,8.663350e+17,4.196984e+09,2017-05-21 16:48:45 +0000,https://twitter.com/dog_rates/status/866334964...,12,10,Coco,None,None,None,None
95,873697596434513921,NaN,NaN,2017-06-11 00:25:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Walter. He won't start ...,8.688804e+17,4.196984e+09,2017-05-28 17:23:24 +0000,https://twitter.com/dog_rates/status/868880397...,14,10,Walter,None,None,None,None
97,873337748698140672,NaN,NaN,2017-06-10 00:35:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Sierra. She's one preci...,8.732138e+17,4.196984e+09,2017-06-09 16:22:42 +0000,https://www.gofundme.com/help-my-baby-sierra-g...,12,10,Sierra,None,None,pupper,None


In [571]:
#Identifying replys: 
twitter_archive[twitter_archive['in_reply_to_status_id'].notnull()== True]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Jack_Septic_Eye I'd need a few more pics to p...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
149,863079547188785154,6.671522e+17,4.196984e+09,2017-05-12 17:12:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Ladies and gentlemen... I found Pipsy. He may ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/863079547...,14,10,None,None,None,None,None
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Marc_IRL pixelated af 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
184,856526610513747968,8.558181e+17,4.196984e+09,2017-04-24 15:13:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...","THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/856526610...,14,10,None,None,None,None,None
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@xianmcguire @Jenna_Marbles Kardashians wouldn...,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None


In [572]:
#droping the retweets and replys: 
twitter_archive_clean.drop(twitter_archive_clean[twitter_archive_clean['retweeted_status_id'].notnull()== True].index,inplace=True)
twitter_archive_clean.drop(twitter_archive_clean[twitter_archive_clean['in_reply_to_status_id'].notnull()== True].index,inplace=True)


### Test

In [573]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                      2097 non-null int64
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     2097 non-null datetime64[ns]
source                        2097 non-null object
text                          2097 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 2094 non-null object
rating_numerator              2097 non-null int64
rating_denominator            2097 non-null int64
name                          2097 non-null object
doggo                         2097 non-null object
floofer                       2097 non-null object
pupper                        2097 non-null object
puppo                         2097 non-null object
date                          20

In [463]:
#We could see that the 'retweeted_status_id', 'retweeted_status_user_id' and 'retweeted_status_timestamp' have zero values. 


##### - Fixing Numerators Manually: 
Since there are only few that need to be changed, we will do it manually: 
    - 786709082849828864 : 75 instaed of 9.75
    - 680494726643068929 : 26 inplace of 11.26
    - 778027034220126208 : 27 inplace of 11.27
    - 883482846933004288 : 5  instead of 13.5
    - 681340665377193984 : 5 instead 9.5
    - 835246439529840640 : 960 intead of 13
    - 716439118184652801 : 50 instaed of  11
    - 666287406224695296 : 1 instaed of 9

### Define
change the numerator for every value in the list above manually using the replace function: We will first search them individually to make sure they are not deleted while dropping NA values earlier: 


### Code

In [574]:
#Chaning the numerators for the list above: 

twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 786709082849828864), 'rating_numerator'] = 9.75
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 680494726643068929), 'rating_numerator'] = 11.26
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 778027034220126208), 'rating_numerator'] = 11.27
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 883482846933004288), 'rating_numerator'] = 13.5
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 681340665377193984), 'rating_numerator'] = 9.5
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 835246439529840640), 'rating_numerator'] = 13
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 716439118184652801), 'rating_numerator'] = 11
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 666287406224695296), 'rating_numerator'] = 9

### Test

In [575]:
twitter_archive_clean.rating_numerator.value_counts()

12.00      486
10.00      436
11.00      414
13.00      287
9.00       154
8.00        98
7.00        52
14.00       38
5.00        33
6.00        32
3.00        19
4.00        16
2.00         9
1.00         4
24.00        1
0.00         1
13.50        1
84.00        1
420.00       1
1776.00      1
60.00        1
44.00        1
144.00       1
88.00        1
11.26        1
11.27        1
121.00       1
9.75         1
99.00        1
204.00       1
45.00        1
165.00       1
80.00        1
Name: rating_numerator, dtype: int64

##### - denominator of following is wrong: 
    - 835246439529840640 : 0 intead of 10
    - 716439118184652801 : 50 instaed of  10
    - 666287406224695296 : 2 instead of 10
    - some tweets have denominators other than 10. 

### Define
update the denominators for the above three tweets manually  and then remove all other tweets that do not have 10 as their denominators. 

### Code

In [576]:
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 835246439529840640), 'rating_denominator'] = 10
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 716439118184652801), 'rating_denominator'] = 10
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 666287406224695296), 'rating_denominator'] = 10

In [578]:
#keep tweets that only have a denominator of 10
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.rating_denominator == 10]

### Test

In [579]:
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 835246439529840640), 'rating_denominator']

Series([], Name: rating_denominator, dtype: int64)

In [580]:
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 716439118184652801), 'rating_denominator']

1202    10
Name: rating_denominator, dtype: int64

In [581]:
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 666287406224695296), 'rating_denominator']

2335    10
Name: rating_denominator, dtype: int64

In [582]:
twitter_archive_clean.rating_denominator.value_counts()

10    2082
Name: rating_denominator, dtype: int64

In [584]:
twitter_archive_clean.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,date,time
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13.0,10,Phineas,None,None,None,None,08/01/2017,16:23
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13.0,10,Tilly,None,None,None,None,08/01/2017,00:17


###### - replace "-" to " " in p1, p2 and p3

### Define
replace the '-' in values of columns p1, p2 and p3 using the .replace() method, and then capitalizing every word in the colums p1-p3 using pandas .title() method. 

### Code

In [585]:
#Removing all '_' in the dog breed's (p1,p2,p3) and leaving them blank. 
image_pred_clean['p1']= image_pred_clean['p1'].str.replace('_', ' ')
image_pred_clean['p2']= image_pred_clean['p2'].str.replace('_', ' ')
image_pred_clean['p3']= image_pred_clean['p3'].str.replace('_', ' ')

In [586]:
#Capitalizing the first letter of each word in the columns. 
image_pred_clean['p1']= image_pred_clean['p1'].str.title()
image_pred_clean['p2']= image_pred_clean['p2'].str.title()
image_pred_clean['p3']= image_pred_clean['p3'].str.title()

### Test

In [587]:
image_pred_clean.sample(4)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1178,737826014890496000,https://pbs.twimg.com/media/Cj1I1fbWYAAOwff.jpg,1,Vizsla,0.990391,True,Rhodesian Ridgeback,0.005605,True,Chesapeake Bay Retriever,0.002869,True
1840,837820167694528512,https://pbs.twimg.com/media/C6CI_jbVAAA3-a1.jpg,1,Golden Retriever,0.887625,True,Labrador Retriever,0.068718,True,Kuvasz,0.030387,True
620,680473011644985345,https://pbs.twimg.com/media/CXGGlzvWYAArPfk.jpg,1,Lakeland Terrier,0.796694,True,West Highland White Terrier,0.138709,True,Norwich Terrier,0.016253,True
1443,775729183532220416,https://pbs.twimg.com/media/CsPxk85XEAAeMQj.jpg,1,Web Site,0.989407,False,Hand-Held Computer,0.002139,False,Menu,0.002115,False


In [588]:
image_pred_clean.shape

(2075, 12)

###### - rename the id column to tweet_id in `twitter_json` table  to match the other tables.


### Define
rename the column name in the `twitter_json` table using panda's rename function

### Code

In [589]:
#Since we are just renaming column name, we do not have to use inplace=True becuase it will return Nonetype. 
twitter_json_clean = twitter_json_clean.rename(columns={'id':'tweet_id'})

### Test

In [590]:
twitter_json_clean.head()

,tweet_id,full_text,retweet_count,favorite_count
0,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,8431,38383
1,892177421306343426,This is Tilly. She's just checking pup on you....,6211,32890
2,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,4111,24785
3,891689557279858688,This is Darla. She commenced a snooze mid meal...,8549,41722
4,891327558926688256,This is Franklin. He would like you to stop ca...,9284,39898


In [591]:
twitter_json_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341 entries, 0 to 2340
Data columns (total 4 columns):
tweet_id          2341 non-null int64
full_text         2341 non-null object
retweet_count     2341 non-null int64
favorite_count    2341 non-null int64
dtypes: int64(3), object(1)
memory usage: 64.1+ KB


###### - tweet_id (838085839343206401) is not about a dog and has 0 retweets. 

### Define
remove the above mentioned tweets as its not about dogs. 

### Code

In [592]:
twitter_json_clean[twitter_json_clean.tweet_id == 838085839343206401]

,tweet_id,full_text,retweet_count,favorite_count
283,838085839343206401,@bragg6of8 @Andy_Pace_ we are still looking fo...,0,147


In [593]:
twitter_json_clean.drop(twitter_json_clean.index[283], inplace=True)

### Test

In [594]:
twitter_json_clean[twitter_json_clean.tweet_id == 838085839343206401]

,tweet_id,full_text,retweet_count,favorite_count


In [595]:
twitter_json_clean.shape

(2340, 4)

## Tidiness:
##### -  doggo, floofer, pupper and puppo should be one column 'stage'
##### - change the datatype of the stage column to categorical. 


### Define
Create a new comlumn that parses the dog stage and put it into one colum. Then remove the existing columns for each dog stage. We will do so using a fuction to help avoide repetition: 


### Code


In [596]:
#first create a list to store the results in it. 
stage = []
#since we arleady know the four dog stages, we will ask the parsere to look only for those. 
a = ['pupper', 'puppo', 'doggo', 'floof']
b = ['pupper', 'puppo', 'doggo', 'floof']
#our for loop will search the text of each tweet. 
for row in twitter_archive_clean['text']:
    row = row.lower()
    for word in a:
        if word in str(row):
            stage.append(b[a.index(word)])
            break
    else:
        stage.append('None')
#convert the list to a column in our dataframe. 
twitter_archive_clean['stage'] = stage

In [598]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2082 entries, 0 to 2355
Data columns (total 20 columns):
tweet_id                      2082 non-null int64
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     2082 non-null datetime64[ns]
source                        2082 non-null object
text                          2082 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 2079 non-null object
rating_numerator              2082 non-null float64
rating_denominator            2082 non-null int64
name                          2082 non-null object
doggo                         2082 non-null object
floofer                       2082 non-null object
pupper                        2082 non-null object
puppo                         2082 non-null object
date                          

In [599]:
twitter_archive_clean.drop(['doggo', 'floofer','pupper','puppo'], axis=1, inplace=True)

### Test

In [600]:
twitter_archive_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,date,time,stage
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13.0,10,Phineas,08/01/2017,16:23,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13.0,10,Tilly,08/01/2017,00:17,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12.0,10,Archie,07/31/2017,00:18,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13.0,10,Darla,07/30/2017,15:58,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12.0,10,Franklin,07/29/2017,16:00,None


In [601]:
twitter_archive_clean.stage.value_counts()

None      1689
pupper     249
doggo       79
floof       35
puppo       30
Name: stage, dtype: int64

In [602]:
twitter_archive_clean.stage = twitter_archive_clean.stage.astype('category')

In [603]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2082 entries, 0 to 2355
Data columns (total 16 columns):
tweet_id                      2082 non-null int64
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     2082 non-null datetime64[ns]
source                        2082 non-null object
text                          2082 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 2079 non-null object
rating_numerator              2082 non-null float64
rating_denominator            2082 non-null int64
name                          2082 non-null object
date                          2082 non-null object
time                          2082 non-null object
stage                         2082 non-null category
dtypes: category(1), datetime64[ns](1), float64(5), int64(2), object(7)
memory 

###### - excessive columns: in_reply_to_status_id, in_reply_to_user_id, source, retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp, timestamp.

### Define
drop the columns mentioned above except timestamp since we already dropped that while converting date and time columns. We coudl do so using the drop function. 

### Code

In [604]:
list(twitter_archive_clean)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'date',
 'time',
 'stage']

In [605]:
twitter_archive_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id',
                           'source','retweeted_status_id', 
                           'retweeted_status_user_id', 
                           'retweeted_status_timestamp', 
                           'expanded_urls',
                           'timestamp'], axis=1, inplace=True)

### Test

In [606]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2082 entries, 0 to 2355
Data columns (total 8 columns):
tweet_id              2082 non-null int64
text                  2082 non-null object
rating_numerator      2082 non-null float64
rating_denominator    2082 non-null int64
name                  2082 non-null object
date                  2082 non-null object
time                  2082 non-null object
stage                 2082 non-null category
dtypes: category(1), float64(1), int64(2), object(4)
memory usage: 99.7+ KB


In [607]:
twitter_archive_clean.head()

,tweet_id,text,rating_numerator,rating_denominator,name,date,time,stage
0,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,13.0,10,Phineas,08/01/2017,16:23,None
1,892177421306343426,This is Tilly. She's just checking pup on you....,13.0,10,Tilly,08/01/2017,00:17,None
2,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,12.0,10,Archie,07/31/2017,00:18,None
3,891689557279858688,This is Darla. She commenced a snooze mid meal...,13.0,10,Darla,07/30/2017,15:58,None
4,891327558926688256,This is Franklin. He would like you to stop ca...,12.0,10,Franklin,07/29/2017,16:00,None


###### - full_text column also exits in twitter_archive, so it needs to be dropped in `twitter_json` table


### Define

### Code

In [610]:

twitter_json_clean.drop('full_text', axis=1, inplace=True)

KeyError: "labels ['full_text'] not contained in axis"

### Test

In [611]:
twitter_json_clean.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8431,38383
1,892177421306343426,6211,32890
2,891815181378084864,4111,24785
3,891689557279858688,8549,41722
4,891327558926688256,9284,39898


In [612]:
twitter_json_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2340 entries, 0 to 2340
Data columns (total 3 columns):
tweet_id          2340 non-null int64
retweet_count     2340 non-null int64
favorite_count    2340 non-null int64
dtypes: int64(3)
memory usage: 73.1 KB



###### - merge the the dog breed (p1-p3) columns. Have only one prediction and its confidence interval in `image_pred` table 
###### - remove columns (img_num-p3_dog) as they will not be needed for the analysis. 
###### - drop rows that does not have a dog breed. 

### Define
combine p1-p2 into one column (dog_breed) and p1_conf-p3_conf to (confid_level) using python's function method. We could do this using the if and elif fuctions. 

### Code

In [613]:
image_pred_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 119.6+ KB


In [614]:
#creat a function that picks the dog breed
#create lists to store prediction of the dog's breed along and its confidence level to be stored. 
dog_breed = []
confid_level= []

#we will name the fuction 'dog_breed_identifier': 
def dog_breed_identifier(image_pred_clean):
    if image_pred_clean['p1_dog']:
        dog_breed.append(image_pred_clean['p1'])
        confid_level.append(image_pred_clean['p1_conf'])
    elif image_pred_clean['p2_dog']:
        dog_breed.append(image_pred_clean['p2'])
        confid_level.append(image_pred_clean['p2_conf'])
    elif image_pred_clean['p3_dog']:
        dog_breed.append(image_pred_clean['p3'])
        confid_level.append(image_pred_clean['p3_conf'])
    else:
        dog_breed.append('Unidentifiable')
        confid_level.append(0)

In [615]:
#Call the function and create new columns: 
image_pred_clean.apply(dog_breed_identifier, axis = 1)
#creating the new columns (dog_breed and confid_level):
image_pred_clean['dog_breed']= dog_breed
image_pred_clean['confid_level'] = confid_level

In [616]:
list(image_pred_clean)

['tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'dog_breed',
 'confid_level']

In [617]:
#Drop the columns that were combined earlier: 
image_pred_clean.drop(['img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',], axis=1, inplace=True)

In [618]:
#now drop rows that their dog type is Unidentifiable:
image_pred_clean = image_pred_clean[image_pred_clean.dog_breed != 'Unidentifiable']

### Test

In [619]:
image_pred_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1751 entries, 0 to 2073
Data columns (total 4 columns):
tweet_id        1751 non-null int64
jpg_url         1751 non-null object
dog_breed       1751 non-null object
confid_level    1751 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 54.7+ KB


In [620]:
image_pred_clean.head()

,tweet_id,jpg_url,dog_breed,confid_level
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,Welsh Springer Spaniel,0.465074
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,Redbone,0.506826
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,German Shepherd,0.596461
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,Rhodesian Ridgeback,0.408143
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,Miniature Pinscher,0.560311


In [624]:
image_pred_clean.dog_breed.value_counts()[:10]

Golden Retriever      173
Labrador Retriever    113
Pembroke               96
Chihuahua              95
Pug                    65
Toy Poodle             52
Chow                   51
Samoyed                46
Pomeranian             42
Cocker Spaniel         34
Name: dog_breed, dtype: int64

###### - twiter_archive and twitter_json should be merged with image_pred table. 

### Define
merge twitter_archive_clean with twitter_json_clean and store them in a new dataframe = twitter_temp using panda's merge function, then merge twitter_master1 with image_pred_clean in the final dataframe called twitter_master using the merge fuction: 


### Code

In [628]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2082 entries, 0 to 2355
Data columns (total 8 columns):
tweet_id              2082 non-null int64
text                  2082 non-null object
rating_numerator      2082 non-null float64
rating_denominator    2082 non-null int64
name                  2082 non-null object
date                  2082 non-null object
time                  2082 non-null object
stage                 2082 non-null category
dtypes: category(1), float64(1), int64(2), object(4)
memory usage: 99.7+ KB


In [629]:
twitter_json_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2340 entries, 0 to 2340
Data columns (total 3 columns):
tweet_id          2340 non-null int64
retweet_count     2340 non-null int64
favorite_count    2340 non-null int64
dtypes: int64(3)
memory usage: 73.1 KB


In [626]:
#Merging twitter_archive_clean with twitter_json_clean: 
twitter_temp = pd.merge(twitter_archive_clean, 
                       twitter_json_clean, 
                       how='left', on=['tweet_id'])

In [627]:
twitter_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2082 entries, 0 to 2081
Data columns (total 10 columns):
tweet_id              2082 non-null int64
text                  2082 non-null object
rating_numerator      2082 non-null float64
rating_denominator    2082 non-null int64
name                  2082 non-null object
date                  2082 non-null object
time                  2082 non-null object
stage                 2082 non-null category
retweet_count         2081 non-null float64
favorite_count        2081 non-null float64
dtypes: category(1), float64(3), int64(2), object(4)
memory usage: 132.3+ KB


In [630]:
twitter_temp.head()

,tweet_id,text,rating_numerator,rating_denominator,name,date,time,stage,retweet_count,favorite_count
0,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,13.0,10,Phineas,08/01/2017,16:23,None,8431.0,38383.0
1,892177421306343426,This is Tilly. She's just checking pup on you....,13.0,10,Tilly,08/01/2017,00:17,None,6211.0,32890.0
2,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,12.0,10,Archie,07/31/2017,00:18,None,4111.0,24785.0
3,891689557279858688,This is Darla. She commenced a snooze mid meal...,13.0,10,Darla,07/30/2017,15:58,None,8549.0,41722.0
4,891327558926688256,This is Franklin. He would like you to stop ca...,12.0,10,Franklin,07/29/2017,16:00,None,9284.0,39898.0


In [631]:
twitter_master = pd.merge(twitter_temp, 
                         image_pred_clean, 
                         how='left', on=['tweet_id'])

In [637]:
#dropping tweets that have null for dog breed. 
twitter_master = twitter_master[twitter_master['dog_breed'].notnull()]
#droppin tweets that have null values in their retweet counts too: 
twitter_master = twitter_master[twitter_master['retweet_count'].notnull()]


### Test

In [638]:
twitter_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1651 entries, 1 to 2081
Data columns (total 13 columns):
tweet_id              1651 non-null int64
text                  1651 non-null object
rating_numerator      1651 non-null float64
rating_denominator    1651 non-null int64
name                  1651 non-null object
date                  1651 non-null object
time                  1651 non-null object
stage                 1651 non-null category
retweet_count         1651 non-null float64
favorite_count        1651 non-null float64
jpg_url               1651 non-null object
dog_breed             1651 non-null object
confid_level          1651 non-null float64
dtypes: category(1), float64(4), int64(2), object(6)
memory usage: 130.7+ KB


In [639]:
twitter_master.sample(10)

,tweet_id,text,rating_numerator,rating_denominator,name,date,time,stage,retweet_count,favorite_count,jpg_url,dog_breed,confid_level
1990,667544320556335104,"This is Kial. Kial is either wearing a cape, w...",10.0,10,Kial,11/20/2015,03:25,None,540.0,884.0,https://pbs.twimg.com/media/CUOYBbbWIAAXQGU.jpg,Pomeranian,0.412893
443,801167903437357056,This is Milo. I would do terrible things for M...,13.0,10,Milo,11/22/2016,20:58,None,6614.0,26487.0,https://pbs.twimg.com/media/Cx5R8wPVEAALa9r.jpg,Cocker Spaniel,0.740220
741,751937170840121344,This is Ruby. Her ice cube is melting. She doe...,11.0,10,Ruby,07/10/2016,00:32,None,1465.0,5561.0,https://pbs.twimg.com/media/Cm9q2d3XEAAqO2m.jpg,Lakeland Terrier,0.424168
1901,669328503091937280,This is Kirk. He just saw a bacon wrapped tenn...,12.0,10,Kirk,11/25/2015,01:35,None,434.0,1034.0,https://pbs.twimg.com/media/CUnuuLEWEAAlKjN.jpg,Siberian Husky,0.424202
2055,666373753744588802,Those are sunglasses and a jean jacket. 11/10 ...,11.0,10,None,11/16/2015,21:54,None,92.0,187.0,https://pbs.twimg.com/media/CT9vZEYWUAAlZ05.jpg,Soft-Coated Wheaten Terrier,0.326467
807,746507379341139972,This is Shiloh. She did not pass the soft mout...,10.0,10,Shiloh,06/25/2016,00:56,None,1172.0,4913.0,https://pbs.twimg.com/media/Clwgf4bWgAAB15c.jpg,Toy Poodle,0.508292
976,716791146589110272,Meet Jennifur. She's supposed to be navigating...,11.0,10,Jennifur,04/04/2016,00:55,None,1489.0,5833.0,https://pbs.twimg.com/media/CfKNvU8WsAAvI9Z.jpg,Pomeranian,0.468751
1589,675822767435051008,🎶 HELLO FROM THE OTHER SIIIIIIIIDE 🎶 10/10s ht...,10.0,10,None,12/12/2015,23:41,None,566.0,1585.0,https://pbs.twimg.com/media/CWEBOFYWwAA-O2c.jpg,Pomeranian,0.460710
1297,689661964914655233,Meet Luca. He's a Butternut Scooperfloof. Glor...,12.0,10,Luca,01/20/2016,04:13,floof,1005.0,3391.0,https://pbs.twimg.com/media/CZIr5gFUsAAvnif.jpg,Italian Greyhound,0.322818
1028,710833117892898816,Say hello to Sora. She's an Egyptian Pumpernic...,12.0,10,Sora,03/18/2016,14:19,None,580.0,2834.0,https://pbs.twimg.com/media/Cd1i8qvUkAE-Jlr.jpg,Pembroke,0.803742


# Storing the dataset:
Now that we have enverything compiled into one main dataset, we can save it in a csv file so that we could use it for our EDA and any other anaylsis in the future. 

In [ ]:
twitter_master.to_csv()

#References: 
- https://stackoverflow.com/questions/35595710/splitting-timestamp-column-into-seperate-date-and-time-columns
- https://stackoverflow.com/questions/45531489/converting-different-date-time-formats-to-mm-dd-yyyy-format-in-pandas-dataframe
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.astype.html
- https://stackoverflow.com/questions/39141856/capitalize-first-letter-of-each-word-in-the-column-python
- https://stackoverflow.com/questions/42478887/why-is-my-pandas-dataframe-turning-into-none-type
- 